In [1]:
!pip install --upgrade pip
!pip install --upgrade git+https://github.com/huggingface/transformers.git accelerate datasets[audio]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-5xf35ev1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-5xf35ev1
  Resolved https://github.com/huggingface/transformers.git to commit 0864dd3beb238b7bec3528a3d1d6c17a28f51a51
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s et

In [1]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

# dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
# sample = dataset[0]["audio"]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
import os

directory_path = "/content/drive/MyDrive/Fall 2023/ECS 289G/Project/Dataset/spliced"
files = []

for filename in os.listdir(directory_path):
    files.append(filename)

sorted_files = sorted(files, key=lambda x: int(x.split('_')[-1].split('.')[0]))
print(sorted_files)

for filename in sorted_files:
    if os.path.isfile(os.path.join(directory_path, filename)):
        file_path = os.path.join(directory_path, filename)
        result = pipe(file_path)
        print("Speaker " + filename.split('_')[1] + ": " + result["text"] + "\n")

['SPEAKER_00_0.wav', 'SPEAKER_00_1.wav', 'SPEAKER_01_2.wav', 'SPEAKER_00_3.wav', 'SPEAKER_01_4.wav', 'SPEAKER_00_5.wav', 'SPEAKER_00_6.wav']
Speaker 00:  you

Speaker 00:  Hey Shreyas, how's your day going?

Speaker 01:  Hi Rohit, my day is going good, how about yours?

Speaker 00:  Oh, that's great. Yeah, even mine's going great. I just got done with my artificial intelligence presentation a while ago, and now I'm working on my project for it. How about you?

Speaker 01:  I am just working on an assignment for my linear algebra course which is due by midnight

Speaker 00:  Oh, great. So, let's see how this audio transcript works in our Hugging Face model.

Speaker 00:  Shud.

